In [1]:
import numpy as np
import pandas as pd
import matplotlib as mp
import csv
import os
import glob

## Step 1: Reformat and Ingest Data

In [2]:
# Combine county files into single dataframe
def merge_files(directory):
    target_files = glob.glob(directory)
    combined_df = pd.DataFrame()
    for file in target_files:
        df = pd.read_table(file, names=['county_code', 'county_name', 'elec_num', 'elec_date', 'elec_name', 'precinct_id', 'poll_loc', 'total_reg', 'total_reg_r', 'total_reg_d', 'total_reg_other', 'contest_name', 'district', 'contest_code', 'cand_or_issue', 'cand_party', 'cand_id', 'doe_num', 'vote_total'], encoding_errors='replace')
        combined_df = pd.concat([combined_df, df])
    return combined_df

In [3]:
location = "C:\\Users\\canor\\Documents\\GitHub\\FL-Political-Analysis\\Florida Analysis\\FL 2016 by Precinct\\*"

df = merge_files(location)

In [4]:
# Examine dataframe for size, dtypes, and NaNs
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 626689 entries, 0 to 1663
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   county_code      626689 non-null  object 
 1   county_name      626689 non-null  object 
 2   elec_num         626689 non-null  int64  
 3   elec_date        626689 non-null  object 
 4   elec_name        626689 non-null  object 
 5   precinct_id      626689 non-null  object 
 6   poll_loc         609703 non-null  object 
 7   total_reg        626689 non-null  int64  
 8   total_reg_r      626689 non-null  int64  
 9   total_reg_d      626689 non-null  int64  
 10  total_reg_other  626689 non-null  int64  
 11  contest_name     626689 non-null  object 
 12  district         614659 non-null  object 
 13  contest_code     626503 non-null  float64
 14  cand_or_issue    626689 non-null  object 
 15  cand_party       605224 non-null  object 
 16  cand_id          621634 non-null  float6

In [5]:
# Examine sample just to get a visual sense of the shape of the data
df.sample(10)

,county_code,county_name,elec_num,elec_date,elec_name,precinct_id,poll_loc,total_reg,total_reg_r,total_reg_d,total_reg_other,contest_name,district,contest_code,cand_or_issue,cand_party,cand_id,doe_num,vote_total
1889,CIT,Citrus,10282,11/08/2016,2016 General Election,305,305 - S Homosassa Springs,5005,0,0,105696,Amendment No. 5: Homestead Tax exemption for C...,"Low-income, Long-term Residents; Determinatio...",900500.0,Yes for Approval,NOP,0.0,10.0,2533
20200,ORA,Orange,10282,11/08/2016,2016 General Election,430,PCT 430,5036,0,0,734553,United States Senator,,120000.0,Marco Rubio,REP,109340956.0,65070.0,2025
43740,HIL,Hillsborough,10282,11/08/2016,2016 General Election,955,955.0,4592,0,0,0,Clerk of the Circuit Court and Comptroller,,320000.0,Pat Frank,DEM,110344452.0,66461.0,1519
6095,CHA,Charlotte,10282,11/08/2016,2016 General Election,63,HOLY TRINITY GREEK 0016,5025,0,0,128332,Retention of Robert Morris,Second District Court of Appeal,520206.0,OverVotes,,0.0,901.0,0
17717,ORA,Orange,10282,11/08/2016,2016 General Election,411,PCT 411,3359,0,0,0,Property Appraiser,,340000.0,Edward DeAguilera,REP,113184439.0,68198.0,1077
53481,DAD,Miami-Dade,10282,11/08/2016,2016 General Election,763,PCT 763 0489,2214,0,0,337369,Representative in Congress,District 26,140260.0,Joe Garcia,DEM,109196349.0,67909.0,531
14412,LEE,Lee,10282,11/08/2016,2016 General Election,99,PCT. 099,3626,0,0,423938,Amendment No. 1: Rights of Electricity Consume...,,900100.0,Yes for Approval,NOP,0.0,10.0,1408
4967,PAL,Palm Beach,10282,11/08/2016,2016 General Election,1142,"1142, 1144",1193,0,0,49419,State Representative,District 82,260820.0,Mary Lynn Magar,REP,105830547.0,64528.0,602
13633,SAR,Sarasota,10282,11/08/2016,2016 General Election,521,521,1177,0,0,301926,Amendment No. 1: Rights of Electricity Consume...,,900100.0,No for Rejection,NOP,0.0,20.0,544
9458,OSC,Osceola,10282,11/08/2016,2016 General Election,571,571 - CANOE CREEK CHARTER SCHO,56,0,0,183586,United States Senator,,120000.0,OverVotes,,0.0,901.0,0


## Step 2: Removing Duplicates

In [8]:
df.duplicated().sum()

154

Looks like we have some dupes! Let's take a look and just make sure we're good to drop them.

In [14]:
df_dupe = df[df.duplicated(keep=False) == True]

,county_code,county_name,elec_num,elec_date,elec_name,precinct_id,poll_loc,total_reg,total_reg_r,total_reg_d,total_reg_other,contest_name,district,contest_code,cand_or_issue,cand_party,cand_id,doe_num,vote_total
32352,DAD,Miami-Dade,10282,11/08/2016,2016 General Election,463,PCT 367/453/463 0231,0,0,0,1307458,President of the United States,,100000.0,Trump / Pence,REP,0.0,65072.0,0
32353,DAD,Miami-Dade,10282,11/08/2016,2016 General Election,463,PCT 367/453/463 0231,0,0,0,1307458,President of the United States,,100000.0,Clinton / Kaine,DEM,0.0,65058.0,0
32354,DAD,Miami-Dade,10282,11/08/2016,2016 General Election,463,PCT 367/453/463 0231,0,0,0,1307458,President of the United States,,100000.0,Johnson / Weld,LPF,0.0,69370.0,0
32355,DAD,Miami-Dade,10282,11/08/2016,2016 General Election,463,PCT 367/453/463 0231,0,0,0,1307458,President of the United States,,100000.0,Castle / Bradley,CPF,0.0,69385.0,0
32356,DAD,Miami-Dade,10282,11/08/2016,2016 General Election,463,PCT 367/453/463 0231,0,0,0,1307458,President of the United States,,100000.0,Stein / Baraka,GRE,0.0,69377.0,0
32357,DAD,Miami-Dade,10282,11/08/2016,2016 General Election,463,PCT 367/453/463 0231,0,0,0,1307458,President of the United States,,100000.0,De La Fuente / Steinberg,REF,0.0,69403.0,0


Upon closer investigations, it looks like all of the dupes are from two precincts in Miami-Dade, and none of the rows register any vote totals -- safe to drop!

In [16]:
df = df.drop_duplicates()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 626535 entries, 0 to 1663
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   county_code      626535 non-null  object 
 1   county_name      626535 non-null  object 
 2   elec_num         626535 non-null  int64  
 3   elec_date        626535 non-null  object 
 4   elec_name        626535 non-null  object 
 5   precinct_id      626535 non-null  object 
 6   poll_loc         609549 non-null  object 
 7   total_reg        626535 non-null  int64  
 8   total_reg_r      626535 non-null  int64  
 9   total_reg_d      626535 non-null  int64  
 10  total_reg_other  626535 non-null  int64  
 11  contest_name     626535 non-null  object 
 12  district         614505 non-null  object 
 13  contest_code     626349 non-null  float64
 14  cand_or_issue    626535 non-null  object 
 15  cand_party       605070 non-null  object 
 16  cand_id          621480 non-null  float6

## Step 3: Removing Extraneous Columns
Having already cleaned the 2012 and 2014 data, I know that there are a handful of columns we can remove off the bat:

In [17]:
df = df.drop(columns=['total_reg_r', 'total_reg_d', 'total_reg_other', 'elec_num', 'elec_name'])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 626535 entries, 0 to 1663
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   county_code    626535 non-null  object 
 1   county_name    626535 non-null  object 
 2   elec_date      626535 non-null  object 
 3   precinct_id    626535 non-null  object 
 4   poll_loc       609549 non-null  object 
 5   total_reg      626535 non-null  int64  
 6   contest_name   626535 non-null  object 
 7   district       614505 non-null  object 
 8   contest_code   626349 non-null  float64
 9   cand_or_issue  626535 non-null  object 
 10  cand_party     605070 non-null  object 
 11  cand_id        621480 non-null  float64
 12  doe_num        626349 non-null  float64
 13  vote_total     626535 non-null  int64  
dtypes: float64(3), int64(2), object(9)
memory usage: 71.7+ MB


## Step 4: Standardizing Contest Names

Since we're only interested in six races (President, Governor, U.S. Senate, State Senate, U.S. Representative, and State Representative) and there was no presidential or U.S. Senate race in Florida in 2014, we need to comb through this list and identify any entries in `contest_name` that suggests it refers to one of those races.

In [20]:
# Look at all listed races and select those that are relevant
df['contest_name'].unique()

array(['President of the United States', 'United States Senator',
       'Representative in Congress', 'State Senator',
       'Clerk of the Circuit Court and Comptroller', 'Sheriff',
       'County Commissioner', 'Retention of Charles T. Canady',
       'Retention of Jorge Labarga', 'Retention of Ricky L. Polston',
       'Retention of Ross Bilbrey', 'Retention of Susan Kelsey',
       'Retention of Lori S. Rowe', 'Retention of Kent Wetherell',
       'Retention of Bo Winokur', 'Retention of Jim Wolf',
       'Amendment No. 1: Rights of Electricity Consumers Regarding Solar Energy Choice',
       'Amendment No. 2: Use of Marijuana for Debilitating Medical Conditions',
       'Amendment No. 3: Tax Exemption for Totally and Permanently Disabled First Responders',
       'Amendment No. 5: Homestead Tax exemption for Certain Senior, Low-income, Long-term Residents; Determination of Just Value',
       'State Representative', 'Tax Collector',
       'Superintendent of Schools', 'Beach Mosq

Above, we can see that 1) there are a lot of races represented in the data and 2) they aren't all uniformly named. Going through the above set of race names, I have extracted the ones that correspond to races we're interested in. Using dictionaries, we can standardize the naming convention for our races.

In [21]:
df['contest_name'] = df['contest_name']\
.replace(dict.fromkeys(['PRESIDENT OF THE UNITED STATES'], 'President of the United States'))\
.replace(dict.fromkeys(['United States Senator', 'UNITED STATES SENATOR'], 'U.S. Senator'))\
.replace(dict.fromkeys(['Congress 10', 'Congress 9', 'Congress 15', 'Congress 17', 'Representative in Congress', 'U.S. REPRESENTATIVE', 'REPRESENTATIVE IN CONGRESS'], 'U.S. Representative'))\
.replace(dict.fromkeys(['GOVERNOR AND  LT.GOVERNOR', 'Governor'], 'Governor and Lieutenant Governor'))\
.replace(dict.fromkeys(['STATE SENATOR', 'Senate 14'], 'State Senator'))\
.replace(dict.fromkeys(['STATE REPRESENTATIVE', 'House 39', 'House 40', 'House 41', 'House 42'], 'State Representative'))



In [22]:
df['contest_name'].unique()

array(['President of the United States', 'U.S. Senator',
       'U.S. Representative', 'State Senator',
       'Clerk of the Circuit Court and Comptroller', 'Sheriff',
       'County Commissioner', 'Retention of Charles T. Canady',
       'Retention of Jorge Labarga', 'Retention of Ricky L. Polston',
       'Retention of Ross Bilbrey', 'Retention of Susan Kelsey',
       'Retention of Lori S. Rowe', 'Retention of Kent Wetherell',
       'Retention of Bo Winokur', 'Retention of Jim Wolf',
       'Amendment No. 1: Rights of Electricity Consumers Regarding Solar Energy Choice',
       'Amendment No. 2: Use of Marijuana for Debilitating Medical Conditions',
       'Amendment No. 3: Tax Exemption for Totally and Permanently Disabled First Responders',
       'Amendment No. 5: Homestead Tax exemption for Certain Senior, Low-income, Long-term Residents; Determination of Just Value',
       'State Representative', 'Tax Collector',
       'Superintendent of Schools', 'Beach Mosquito Control Dis

## Step 5: Narrowing to Races
And now we can more confidently narrow to our four races:

In [23]:
df = df[df.contest_name.isin(['President of the United States',\
                              'U.S. Senator',\
                              'U.S. Representative',\
                              'Governor and Lieutenant Governor',\
                              'State Senator',\
                              'State Representative'])]\
                            .reset_index()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165059 entries, 0 to 165058
Data columns (total 15 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   index          165059 non-null  int64  
 1   county_code    165059 non-null  object 
 2   county_name    165059 non-null  object 
 3   elec_date      165059 non-null  object 
 4   precinct_id    165059 non-null  object 
 5   poll_loc       159720 non-null  object 
 6   total_reg      165059 non-null  int64  
 7   contest_name   165059 non-null  object 
 8   district       160366 non-null  object 
 9   contest_code   165059 non-null  float64
 10  cand_or_issue  165059 non-null  object 
 11  cand_party     162347 non-null  object 
 12  cand_id        163755 non-null  float64
 13  doe_num        165059 non-null  float64
 14  vote_total     165059 non-null  int64  
dtypes: float64(3), int64(3), object(9)
memory usage: 18.9+ MB


## Step 6: Dealing With NaN Cells

In [24]:
# Looking for any rows with nulls
df_nan = df[df.isna().any(axis=1)]

df_nan

,index,county_code,county_name,elec_date,precinct_id,poll_loc,total_reg,contest_name,district,contest_code,cand_or_issue,cand_party,cand_id,doe_num,vote_total
2185,0,BAY,Bay,11/08/2016,1,NaN,2003,President of the United States,,100000.0,Trump / Pence,REP,0.0,65072.0,1122
2186,1,BAY,Bay,11/08/2016,1,NaN,2003,President of the United States,,100000.0,Clinton / Kaine,DEM,0.0,65058.0,189
2187,2,BAY,Bay,11/08/2016,1,NaN,2003,President of the United States,,100000.0,Johnson / Weld,LPF,0.0,69370.0,25
2188,3,BAY,Bay,11/08/2016,1,NaN,2003,President of the United States,,100000.0,Castle / Bradley,CPF,0.0,69385.0,0
2189,4,BAY,Bay,11/08/2016,1,NaN,2003,President of the United States,,100000.0,Stein / Baraka,GRE,0.0,69377.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154706,8113,SEM,SEMINOLE,11/8/2016,76,Precinct 76,4117,U.S. Senator,NaN,120000.0,Write-in,NaN,NaN,900.0,5
154707,8114,SEM,SEMINOLE,11/8/2016,77,Precinct 77,2782,U.S. Senator,NaN,120000.0,Write-in,NaN,NaN,900.0,3
154708,8115,SEM,SEMINOLE,11/8/2016,78,Precinct 78,4173,U.S. Senator,NaN,120000.0,Write-in,NaN,NaN,900.0,6
154709,8116,SEM,SEMINOLE,11/8/2016,79,Precinct 79,3698,U.S. Senator,NaN,120000.0,Write-in,NaN,NaN,900.0,9


In [25]:
# Making sure rows with nulls in 'cand_party' are only under and overvotes, which we can discard
df_nan_cand_party = df[df['cand_party'].isna()]

df_nan_cand_party

,index,county_code,county_name,elec_date,precinct_id,poll_loc,total_reg,contest_name,district,contest_code,cand_or_issue,cand_party,cand_id,doe_num,vote_total
143191,1002,POL,Polk,11/08/2016,101,Outreach Baptist,2577,President of the United States,NaN,100000.0,WriteInVotes,NaN,0.0,0.0,17
143192,1003,POL,Polk,11/08/2016,102,Green Pond Baptist Church,1149,President of the United States,NaN,100000.0,WriteInVotes,NaN,0.0,0.0,2
143193,1004,POL,Polk,11/08/2016,103,Polo Park,4043,President of the United States,NaN,100000.0,WriteInVotes,NaN,0.0,0.0,22
143194,1005,POL,Polk,11/08/2016,104,Polo Park East,2901,President of the United States,NaN,100000.0,WriteInVotes,NaN,0.0,0.0,14
143195,1006,POL,Polk,11/08/2016,105,Ridge Mobile Home Park Clubhouse,1752,President of the United States,NaN,100000.0,WriteInVotes,NaN,0.0,0.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154706,8113,SEM,SEMINOLE,11/8/2016,76,Precinct 76,4117,U.S. Senator,NaN,120000.0,Write-in,NaN,NaN,900.0,5
154707,8114,SEM,SEMINOLE,11/8/2016,77,Precinct 77,2782,U.S. Senator,NaN,120000.0,Write-in,NaN,NaN,900.0,3
154708,8115,SEM,SEMINOLE,11/8/2016,78,Precinct 78,4173,U.S. Senator,NaN,120000.0,Write-in,NaN,NaN,900.0,6
154709,8116,SEM,SEMINOLE,11/8/2016,79,Precinct 79,3698,U.S. Senator,NaN,120000.0,Write-in,NaN,NaN,900.0,9


In [26]:

# Making sure rows with nulls in 'cand_party' are only under and overvotes, which we can discard
df_nan_cand_party['cand_or_issue'].unique()

array(['WriteInVotes', 'OverVotes', 'UnderVotes', 'Times Blank Voted',
       'Times Over Voted', 'Write-in 30', 'Write-in'], dtype=object)

Looks like rows containing NaNs in cand_party are either rows describing overvotes, undervotes, or write-ins,- or pertain to nonpartisan races that are not relevant to this analysis. Let's double check that latter assumption, and then drop those rows:

In [27]:
df_nan_cand_party[~df_nan_cand_party['cand_or_issue'].isin(['UnderVotes', 'OverVotes'])]['contest_name'].unique()

array(['President of the United States', 'U.S. Senator',
       'U.S. Representative', 'State Representative'], dtype=object)

Assumptions confirmed. We can safely drop any rows that have a NaN value in the `cand_party` column.

In [ ]:
df = df.dropna(subset=['cand_party'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81637 entries, 0 to 82602
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          81637 non-null  int64  
 1   county_code    81637 non-null  object 
 2   county_name    81637 non-null  object 
 3   elec_date      81637 non-null  object 
 4   precinct_id    81637 non-null  object 
 5   poll_loc       72295 non-null  object 
 6   total_reg      81637 non-null  int64  
 7   contest_name   81637 non-null  object 
 8   district       75950 non-null  object 
 9   contest_code   81637 non-null  int64  
 10  cand_or_issue  81637 non-null  object 
 11  cand_party     81637 non-null  object 
 12  cand_id        80910 non-null  float64
 13  doe_num        81637 non-null  int64  
 14  vote_total     81637 non-null  int64  
dtypes: float64(1), int64(5), object(9)
memory usage: 10.0+ MB


Looks like we still have nulls in `poll_loc` (which is fine), `district`, and `cand_id` -- let's take a look at those.

In [ ]:
# Create a df with just the rows containing a null value in the district column
df_nan_district = df[df['district'].isna()]
df_nan_district.head(5)

,index,county_code,county_name,elec_date,precinct_id,poll_loc,total_reg,contest_name,district,contest_code,cand_or_issue,cand_party,cand_id,doe_num,vote_total
57918,1477,PAL,Palm Beach,11/4/2014,1002,1002,0,Governor and Lieutenant Governor,NaN,160000,Adrian Wyllie,LPF,106930498.0,60196,0
57919,1478,PAL,Palm Beach,11/4/2014,1004,1004,2,Governor and Lieutenant Governor,NaN,160000,Adrian Wyllie,LPF,106930498.0,60196,0
57920,1479,PAL,Palm Beach,11/4/2014,1006,1006,0,Governor and Lieutenant Governor,NaN,160000,Adrian Wyllie,LPF,106930498.0,60196,0
57921,1480,PAL,Palm Beach,11/4/2014,1008,1008,0,Governor and Lieutenant Governor,NaN,160000,Adrian Wyllie,LPF,106930498.0,60196,0
57922,1481,PAL,Palm Beach,11/4/2014,1010,1010,0,Governor and Lieutenant Governor,NaN,160000,Adrian Wyllie,LPF,106930498.0,60196,0


This is a little weird. Looks like we have NaNs in the district column for Governor and Lt. Governor candidates, in rows where vote tallies are indeed present. Is this just for the libertarian party candidate (as we can see in the above head window)? Let's check.

In [ ]:
# Is this only present for the Governor's race? Doesn't look like it.
df_nan_district['contest_name'].unique()

array(['Governor and Lieutenant Governor'], dtype=object)

In [ ]:
# Is this only present in Palm Beach? Doesn't look like it.
df_nan_district['county_name'].unique()

array(['Palm Beach', 'Polk', 'Seminole'], dtype=object)

In [ ]:
# Is this only present for LPF party candidates? Doesn't look like it.
df_nan_district['cand_party'].unique()

array(['LPF', 'DEM', 'NPA', 'REP', 'NP'], dtype=object)

It looks like the only races that we're interested in that have null values in the `district` column are statewide and don't really have a district anyway. We'll just replace these NaNs with blank cells and move on.

In [ ]:
# We can fill NaNs in this column for rows for statewide races with 'Statewide' be consistent with other dataframes.
df['district'].fillna('', inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81637 entries, 0 to 82602
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          81637 non-null  int64  
 1   county_code    81637 non-null  object 
 2   county_name    81637 non-null  object 
 3   elec_date      81637 non-null  object 
 4   precinct_id    81637 non-null  object 
 5   poll_loc       72295 non-null  object 
 6   total_reg      81637 non-null  int64  
 7   contest_name   81637 non-null  object 
 8   district       81637 non-null  object 
 9   contest_code   81637 non-null  int64  
 10  cand_or_issue  81637 non-null  object 
 11  cand_party     81637 non-null  object 
 12  cand_id        80910 non-null  float64
 13  doe_num        81637 non-null  int64  
 14  vote_total     81637 non-null  int64  
dtypes: float64(1), int64(5), object(9)
memory usage: 10.0+ MB


Okay, we're at a point where the only remaining nulls are in `cand_id` and `poll_loc`, neither of which are really vital. We can move on from filling in NaNs.

## Step 7: Check for Completeness
Finally, let's just double check to make sure all counties and races are present in our final output:

In [ ]:
counties = df['county_name'].unique()
print(counties)
print(len(counties))
# All counties are present and accounted for 

['Alachua' 'Baker' 'Bay' 'Bradford' 'Brevard' 'Broward' 'Calhoun'
 'Charlotte' 'Citrus' 'Clay' 'Collier' 'Columbia' 'Miami-Dade' 'Desoto'
 'Dixie' 'Duval' 'Escambia' 'Flagler' 'Franklin' 'Gadsden' 'Gilchrist'
 'Glades' 'Gulf' 'Hamilton' 'Hardee' 'Hendry' 'Hernando' 'Highlands'
 'Hillsborough' 'Holmes' 'Indian River' 'Jackson' 'Jefferson' 'Lafayette'
 'Lake' 'Lee' 'Leon' 'Levy' 'Liberty' 'Madison' 'Manatee' 'Monroe'
 'Marion' 'Martin' 'Nassau' 'Okaloosa' 'Okeechobee' 'Orange' 'Osceola'
 'Palm Beach' 'Pasco' 'Pinellas' 'Polk' 'Putnam' 'Santa Rosa' 'Sarasota'
 'Seminole' 'St. Johns' 'St. Lucie' 'Sumter' 'Suwannee' 'Taylor' 'Union'
 'Volusia' 'Wakulla' 'Walton' 'Washington']
67


All 67 counties are represented in the dataset, so this looks complete.

In [ ]:
races = df['contest_name'].unique()
print(races)
print(len(races))

['U.S. Representative' 'Governor and Lieutenant Governor'
 'State Representative' 'State Senator']
4


2014 had no presidential contest or senate races, so this looks complete.

## Step 8: Save to CSV

In [7]:
# df.to_csv('fl_2016_cleaned.csv')